In [5]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [6]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [7]:
input_prefix = 'data/'
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/data/'

bzfile = '.bz2' # ''

In [8]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
df_train = pd.read_csv(\
    input_prefix + 'sales_train.csv' + bzfile,
    #nrows=300, # somente primeiras linhas!
    parse_dates=['date'], date_parser=dateparse,
    dtype={'date_block_num': np.int8, 'shop_id': np.int8, 'item_id': np.int16, \
           'item_price': np.float32, 'item_cnt_day': np.int32},
    )
print('shape:', df_train.shape)
df_train.head()

shape: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1
1,2013-01-03,0,25,2552,899.00,1
2,2013-01-05,0,25,2552,899.00,-1
3,2013-01-06,0,25,2554,1709.05,1
4,2013-01-15,0,25,2555,1099.00,1


In [9]:
%time df_train['item_price'] = (np.ceil(df_train['item_price'])).astype(np.int32)

CPU times: user 32.9 ms, sys: 0 ns, total: 32.9 ms
Wall time: 31.5 ms


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              datetime64[ns]
date_block_num    int8
shop_id           int8
item_id           int16
item_price        int32
item_cnt_day      int32
dtypes: datetime64[ns](1), int16(1), int32(2), int8(2)
memory usage: 56.0 MB


In [11]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06
mean,1.4570e+01,3.3002e+01,1.0197e+04,8.9089e+02,1.2426e+00
std,9.4230e+00,1.6227e+01,6.3243e+03,1.7298e+03,2.6188e+00
min,0.0000e+00,0.0000e+00,0.0000e+00,-1.0000e+00,-2.2000e+01
25%,7.0000e+00,2.2000e+01,4.4760e+03,2.4900e+02,1.0000e+00
50%,1.4000e+01,3.1000e+01,9.3430e+03,3.9900e+02,1.0000e+00
75%,2.3000e+01,4.7000e+01,1.5684e+04,9.9900e+02,1.0000e+00
max,3.3000e+01,5.9000e+01,2.2169e+04,3.0798e+05,2.1690e+03


In [12]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999,1
1,2013-01-03,0,25,2552,899,1
2,2013-01-05,0,25,2552,899,-1
3,2013-01-06,0,25,2554,1710,1
4,2013-01-15,0,25,2555,1099,1


In [13]:
mins = df_train.groupby(['date_block_num'])['date'].min()
maxs = df_train.groupby(['date_block_num'])['date'].max()

date_blocks = pd.concat([mins, maxs], axis=1)
date_blocks

,date,date
date_block_num,,
0,2013-01-01,2013-01-31
1,2013-02-01,2013-02-28
2,2013-03-01,2013-03-31
3,2013-04-01,2013-04-30
4,2013-05-01,2013-05-31
5,2013-06-01,2013-06-30
6,2013-07-01,2013-07-31
7,2013-08-01,2013-08-31
8,2013-09-01,2013-09-30


In [22]:
grouped_data = df_train.groupby(['date_block_num'], as_index=True).agg(
    {'date': ['min', 'max', 'count']}
)

grouped_data.sort_values(by=('date', 'min'), ascending=True).head(15)

date                   
                      min        max   count
date_block_num                              
0              2013-01-01 2013-01-31  115690
1              2013-02-01 2013-02-28  108613
2              2013-03-01 2013-03-31  121347
3              2013-04-01 2013-04-30   94109
4              2013-05-01 2013-05-31   91759
5              2013-06-01 2013-06-30  100403
6              2013-07-01 2013-07-31  100548
7              2013-08-01 2013-08-31  104772
8              2013-09-01 2013-09-30   96137
9              2013-10-01 2013-10-31   94202
10             2013-11-01 2013-11-30   96736
11             2013-12-01 2013-12-31  143246
12             2014-01-01 2014-01-31   99349
13             2014-02-01 2014-02-28   89830
14             2014-03-01 2014-03-31   92733